In [1]:
# GitHub Link: https://github.com/Lasata-T/Netflix_Project.git

In [7]:
# imports
import csv
import pandas as pd
import os
import datetime
import pymongo
import pprint

In [9]:
!pip install tensorflow --upgrade --force-reinstall

  Using cached tensorflow-2.11.0-cp38-cp38-macosx_10_14_x86_64.whl (244.3 MB)
  Using cached absl_py-1.3.0-py3-none-any.whl (124 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.1 MB/s eta 0:00:0000:0100:01
  Using cached termcolor-2.1.1-py3-none-any.whl (6.2 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 980.4/980.4 kB 14.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 23.3 MB/s eta 0:00:0000:0100:01
  Using cached libclang-14.0.6-py2.py3-none-macosx_10_9_x86_64.whl (13.2 MB)
  Using cached grpcio-1.51.1-cp38-cp38-macosx_10_10_x86_64.whl (4.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.5 MB/s eta 0:00:00
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached numpy-1.23.5-cp38-cp38-macosx_10_9_x86_64.whl (18.1 MB)
  Using cached flatbuffers-22.12.6-py2.py3-none-any.whl (26 kB)
 

  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 49.2.0.post20200714
    Uninstalling setuptools-49.2.0.post20200714:
      Successfully uninstalled setuptools-49.2.0.post20200714
  Attempting uninstall: rsa
    Found existing installation: rsa 4.7.2
    Uninstalling rsa-4.7.2:
      Successfully uninstalled rsa-4.7.2
  Attempting uninstall: pyasn1-modules
    Found existing installation: pyasn1-modules 0.2.8
    Uninstalling pyasn1-modules-0.2.8:
      Successfully uninstalled pyasn1-modules-0.2.8
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: packaging
    Found existing installation: packaging 20.4
    Uninstalling packaging-20.4:
      Successfully uninstalled packaging-20.4

In [10]:
# start chatbot

import nltk 
# import discord
nltk.download('punkt')

from nltk import word_tokenize,sent_tokenize

from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
#read more on the steamer https://towardsdatascience.com/stemming-lemmatization-what-ba782b7c0bd8
import numpy as np 
import tflearn
import tensorflow as tf
import random
import json
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/maniktuladhar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Instructions for updating:
non-resource variables are not supported in the long term


In [12]:
with open("intents-Copy1.json") as file: # change file name later
    data = json.load(file)

In [13]:
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []
    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
            
        if intent["tag"] not in labels:
            labels.append(intent["tag"])


    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))
    labels = sorted(labels)

    training = []
    output = []
    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
               bag.append(1)
            else:
              bag.append(0)
    
        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1
        
        training.append(bag)
        output.append(output_row)

    training = np.array(training)
    output = np.array(output)
    
    with open("data.pickle","wb") as f:
        pickle.dump((words, labels, training, output), f)

In [14]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [15]:
model = tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 1999  | total loss: 0.06124 | time: 0.001s
| Adam | epoch: 1000 | loss: 0.06124 - acc: 0.9893 -- iter: 08/14
Training Step: 2000  | total loss: 0.05662 | time: 0.003s
| Adam | epoch: 1000 | loss: 0.05662 - acc: 0.9904 -- iter: 14/14
--
INFO:tensorflow:/Users/maniktuladhar/Desktop/ds2002-1/NEW_FOLDER/netflix project/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [16]:
try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")

INFO:tensorflow:Restoring parameters from /Users/maniktuladhar/Desktop/ds2002-1/NEW_FOLDER/netflix project/model.tflearn


In [17]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    
    return np.array(bag)

In [ ]:
def chat():
    print("Start talking with the bot! (type quit to stop)")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        result = model.predict([bag_of_words(inp, words)])[0]
        result_index = np.argmax(result)
        tag = labels[result_index]

        if result[result_index] > 0.7:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
            print(random.choice(responses))

        else:
            print("I didnt get that. Can you explain or try again.")
chat()

Start talking with the bot! (type quit to stop)
You: Hi
hurry up, I don't have all day
You: Hello
Good to see you again!
You: Bye
Sad to see you go..


In [1]:
x = 3